In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scam_job_detector.ML_logic.data import clean_data
from scam_job_detector.ML_logic.preprocessor import train_preprocessor, test_preprocessor

ImportError: cannot import name 'test_preprocessor' from 'scam_job_detector.ML_logic.preprocessor' (/Users/gillestondeur/.pyenv/versions/3.10.6/envs/scam_job_detector/lib/python3.10/site-packages/scam_job_detector/ML_logic/preprocessor.py)

In [2]:
df = pd.read_csv("/Users/gillestondeur/code/syeda-tabassum-rahaman/scam-job-detector/raw_data/fake_job_postings.csv")

In [3]:
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [7]:
df.function.nunique()

37

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15184 non-null  object
 8   benefits             10668 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

In [9]:
df.isnull().sum()


job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2696
benefits                7212
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [5]:
df['fraudulent'].value_counts(normalize=True)

fraudulent
0    0.951566
1    0.048434
Name: proportion, dtype: float64

# proportion of fraudulent vs non-fraudulent job postings skwewed --> address to handle class imbalance

In [6]:
req =df.location.nunique()
req

3105

In [7]:
df.sample(100)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
15315,15316,English Teacher Abroad,"CA, ON, Mississauga",NaN,NaN,We help teachers get safe &amp; secure jobs ab...,"Play with kids, get paid for it Love travel? J...",University degree required. TEFL / TESOL / CEL...,See job description,0,1,1,Contract,NaN,Bachelor's Degree,Education Management,NaN,0
17126,17127,Internal Recruiter,"GB, LND,",Operations,NaN,The Beans Group is a media company that exists...,Award-Winning London-based Media Company seek ...,EssentialExcellent communication and negotiati...,Life at The Beans GroupThere are many things t...,0,1,0,Full-time,Associate,Bachelor's Degree,Marketing and Advertising,Human Resources,0
10463,10464,Legal Assistant,"US, IA, Coralville",NaN,NaN,Hawkeye Recruitment provides cost effective re...,Coralville law office has an opening for full-...,"Working skills in the areas of real estate, pr...",NaN,0,1,0,Full-time,Entry level,High School or equivalent,Law Practice,Legal,0
11225,11226,Linux Systems Engineer,"GR, I, Maroussi",IT Engineering Infrastructure,NaN,"inAccess’ InSolar is an integrated, vendor ind...",IT Engineering Infrastructure is looking for L...,5+ years experience with Linuxuniversity Degre...,NaN,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Renewables & Environment,Information Technology,0
14843,14844,SAP ABAP,"US, MA, Waltham",NaN,NaN,NaN,"2 SAP ABAP DevelopersWaltham, MAMust have stro...",NaN,NaN,0,0,0,Contract,Mid-Senior level,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4979,4980,Customer Happiness Technician (cell phone & ta...,"US, WA, Seattle",NaN,NaN,NaN,Local start-up See Jane Fix is seeking experts...,ResponsibilitiesEngage with clients in a posit...,TBD,0,0,0,NaN,Entry level,NaN,NaN,NaN,0
3033,3034,Escape the City Job Fair,NaN,NaN,NaN,NaN,We met at Escape the CityReason 1: Job hunting...,NaN,NaN,0,1,1,NaN,NaN,NaN,NaN,NaN,0
9386,9387,Graduates: English Teacher Abroad,"US, OH, Akron",NaN,NaN,We help teachers get safe &amp; secure jobs ab...,"Play with kids, get paid for it :-)Love travel...",University degree required. TEFL / TESOL / CEL...,See job description,0,1,0,Contract,NaN,Bachelor's Degree,Education Management,NaN,0
10559,10560,Cargo Representative / Loss Control (Petroleum),"US, TX, Houston",NaN,NaN,"PetroCargo LLC, is a specialist in Fuels Asset...","PetroCargo LLC, a specialist in Fuels Asset Ma...",SAMPLE RESPONSIBILITIES:Establish yourself as ...,Base Salary + Participation in our Vessel Perf...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Oil & Energy,Quality Assurance,0


In [8]:
temp = df.has_questions.value_counts()
temp

has_questions
0    9088
1    8792
Name: count, dtype: int64

In [9]:
X = df.drop(columns=['fraudulent'])
y = df['fraudulent']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [11]:
# catagoreical columsn for One-Hot Encoding
categorical_columns = [
    'country',
    'industry',
    'function',
    'employment_type',
]
ordinal_columns = [
    'required_experience',
    'required_education'
]

In [12]:
required_education_values = pd.DataFrame(df.required_education.unique()).iloc[:,0].tolist()
required_experience_values = pd.DataFrame(df.required_experience.unique()).iloc[:,0].tolist()


In [13]:
required_experience_values

['Internship',
 'Not Applicable',
 nan,
 'Mid-Senior level',
 'Associate',
 'Entry level',
 'Executive',
 'Director']

In [29]:
required_education_values

[nan,
 "Bachelor's Degree",
 "Master's Degree",
 'High School or equivalent',
 'Unspecified',
 'Some College Coursework Completed',
 'Vocational',
 'Certification',
 'Associate Degree',
 'Professional',
 'Doctorate',
 'Some High School Coursework',
 'Vocational - Degree',
 'Vocational - HS Diploma']

In [14]:
experience_order = [
    "Not Applicable",
    "Unknown",
    "Internship",
    "Entry level",
    "Associate",
    "Mid-Senior level",
    "Director",
    "Executive"
]

education_order = [
    "Unknown",
    "High School or equivalent",
    "Vocational",
    "Certification",
    "Some College Coursework Completed",
    "Associate Degree",
    "Bachelor's Degree",
    "Professional",
    "Master's Degree"
]
binary_columns = ['has_company_logo', 'has_questions']

In [15]:
cat_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OneHotEncoder(handle_unknown='ignore')
)
ordinal_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OrdinalEncoder(
    categories=[experience_order, education_order],
    handle_unknown="use_encoded_value",
    unknown_value=-1)
)
binary_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent', fill_value=0),
    OneHotEncoder(handle_unknown='ignore')
)
text_transformer = make_pipeline(
    TfidfVectorizer(max_features=5000)
)
preprocessor = make_column_transformer(
    (cat_transformer, categorical_columns),
    (ordinal_transformer, ordinal_columns),
    (binary_transformer, binary_columns),
    (text_transformer, 'TBD') # Placeholder for text column name
)

In [16]:
preprocessing = make_pipeline(
    cat_transformer,
    ordinal_transformer,
    text_transformer
)

In [19]:
# Download raw data
df = pd.read_csv("../raw_data/fake_job_postings.csv")

# Clean data function
df = clean_data(df)

# Extract X and y
X = df.drop(columns=['fraudulent'])
y = df['fraudulent']

# Make train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# preprocess train and test data
X_train = train_preprocessor(X_train)
X_test = test_preprocessor(X_test)

dataset loaded
✅ data cleaned


NameError: name 'train_preprocessor' is not defined